In [1]:
import glob


task_results = glob.glob("/home/gautier/Desktop/plancraft/outputs/2024-02-15/22-32-07/*/*.json")
goal_histories = glob.glob("/home/gautier/Desktop/plancraft/outputs/2024-02-15/22-32-07/*/*.jsonl")

In [2]:
import json
from collections import defaultdict

task_data = {}
for task_result in task_results:
    with open(task_result, "r") as f:
        n = task_result.split("/")[-2]
        task_data[n] = json.load(f)

In [3]:
import pandas as pd
from statistics import mean

results = []
# goal_histories_data
for task, data in task_data.items():
    print(task)
    print(len(data["episode_lengths"]))
    print(data)
    results.append(
        {
            "task": task,
            "success_count": data["success_count"],
            "num_evals": data["num_evals"],
            "success_rate": data["success_rate"],
            "average_episode_length": mean(data["episode_lengths"]),
        }
    )

df = pd.DataFrame(results)
df

obtain_wooden_slab
26
{'success_count': 26, 'success_rate': 0.8666666666666667, 'num_evals': 30, 'episode_lengths': [1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 2999, 1962, 1962, 1962, 1962, 2999, 1962, 612, 1362, 1962, 1962, 1962, 1962, 1962]}
obtain_planks
24
{'success_count': 24, 'success_rate': 0.8, 'num_evals': 30, 'episode_lengths': [1962, 2999, 514, 353, 1962, 349, 1962, 1962, 1147, 1962, 532, 1962, 311, 1158, 1962, 1962, 576, 1205, 1962, 1962, 1951, 1962, 2414, 983]}
obtain_wooden_button
23
{'success_count': 23, 'success_rate': 0.7666666666666667, 'num_evals': 30, 'episode_lengths': [1962, 1212, 2999, 762, 1962, 2999, 1962, 996, 1962, 632, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962]}
obtain_sign
27
{'success_count': 27, 'success_rate': 0.9, 'num_evals': 30, 'episode_lengths': [1962, 1962, 2362, 762, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1962, 1562, 1962, 1962, 2580, 1812,

/tmp/ipykernel_12706/812973444.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,task,success_count,num_evals,success_rate,average_episode_length
0,obtain_wooden_slab,26,30,0.866667,1966.769231
1,obtain_planks,24,30,0.800000,1503.083333
2,obtain_wooden_button,23,30,0.766667,1867.565217
3,obtain_sign,27,30,0.900000,1934.888889
4,obtain_wooden_pressure_plate,26,30,0.866667,1972.115385
5,obtain_chest,24,30,0.800000,2051.000000
6,obtain_stick,26,30,0.866667,1662.461538
7,obtain_oak_stairs,26,30,0.866667,1962.000000


In [4]:
goal_histories_data = defaultdict(dict)
for goal_history in goal_histories:
    with open(goal_history, "r") as f:
        name = goal_history.split("/")[-2]
        iteration = goal_history.split("/")[-1].split("_")[-3]
        goal_histories_data[name][int(iteration)] = []
        for line in f:
            goal_histories_data[name][int(iteration)].append(json.loads(line))

In [5]:
for task in goal_histories_data.keys():
    for iteration in goal_histories_data[task].keys():
        print(task, iteration, len(goal_histories_data[task][iteration]))

obtain_wooden_slab 20 613
obtain_wooden_slab 21 1363
obtain_wooden_slab 13 3000
obtain_wooden_slab 6 1963
obtain_wooden_slab 25 1963
obtain_wooden_slab 14 1963
obtain_wooden_slab 12 1963
obtain_wooden_slab 19 1963
obtain_wooden_slab 5 1963
obtain_wooden_slab 18 3000
obtain_wooden_slab 22 1963
obtain_wooden_slab 29 1963
obtain_wooden_slab 3 1963
obtain_wooden_slab 4 1963
obtain_wooden_slab 11 1963
obtain_wooden_slab 26 1963
obtain_wooden_slab 10 1963
obtain_wooden_slab 2 1963
obtain_wooden_slab 8 1963
obtain_wooden_slab 17 1963
obtain_wooden_slab 16 1963
obtain_wooden_slab 15 1963
obtain_wooden_slab 7 1963
obtain_wooden_slab 9 1963
obtain_wooden_slab 27 1963
obtain_wooden_slab 0 1963
obtain_planks 6 354
obtain_planks 27 1963
obtain_planks 8 1963
obtain_planks 19 1963
obtain_planks 10 1963
obtain_planks 0 1963
obtain_planks 24 1963
obtain_planks 2 515
obtain_planks 17 312
obtain_planks 9 350
obtain_planks 22 1206
obtain_planks 21 577
obtain_planks 16 1963
obtain_planks 29 984
obtain_plan

In [6]:
goal_history.split("/")[-1].split("_")

['obtain', 'oak', 'stairs', '5', 'goal', 'history.jsonl']

In [7]:
# read full log file: /home/gautier/Desktop/plancraft/outputs/2024-02-15/22-32-07/main.log
main_log_file = "/home/gautier/Desktop/plancraft/outputs/2024-02-15/22-32-07/main.log"

with open(main_log_file, "r") as f:
    text = f.readlines()


data_logs = defaultdict(list)
logs_for_task_at_iteration = []
task = None
for line in text:
    if "Evaluating the task is " in line:
        if task:
            data_logs[task].append(logs_for_task_at_iteration)

        task = line.split("Evaluating the task is ")[1].replace("\n", "")     
        logs_for_task_at_iteration = []

    logs_for_task_at_iteration.append(line)

In [8]:
log_results = defaultdict(dict)
for task in data_logs.keys():
    success_count = 0
    finished_goals = []
    planning_error = 0
    parsing_error = 0
    for iteration in range(len(data_logs[task])):
        achieved_goals = []
        for line in data_logs[task][iteration]:
            if "finish goal" in line:
                goal = line.split("finish goal ")[1].replace("\n", "").replace(".", "")
                achieved_goals.append(goal)
                if goal.replace("craft", "obtain") == task:
                    success_count += 1

            if "replanning over" in line:
                planning_error += 1
                break
            if "parsed goal is not supported by current controller" in line:
                parsing_error += 1
                
        finished_goals.append(achieved_goals)

    log_results[task]["goals"] = finished_goals
    log_results[task]["planning_error"] = planning_error
    log_results[task]["success_count"] = success_count
    log_results[task]["parsing_error"] = parsing_error

In [16]:
# data_logs[task][iteration]

df_data = []
for task in log_results.keys():
    df_data.append(
        {
            "task": task.replace("obtain_", ""),
            "success_count": log_results[task]["success_count"],
            "planning_error": log_results[task]["planning_error"],
            "parsing_error": log_results[task]["parsing_error"],
            "success_rate": log_results[task]["success_count"] / len(data_logs[task]),
        }
    )

df = pd.DataFrame(df_data)

df.to_csv("results.csv", index=False)
